In [8]:
import pandas as pd
import re

df_nat = pd.read_csv("resultados_tarifas_naturgy.csv")
df_octopus = pd.read_csv("resultados_tarifas_octopus.csv")
df_repsol = pd.read_csv("resultados_tarifas_repsol_unificado.csv")
df_xenera = pd.read_csv("resultados_tarifas_xenera_unificado.csv")

# Potencia
#P1 Periodo Punta 
#P2 Periodo Llano
#P3 Periodo Valle


# Limpiamos nombres en Octopus
df_octopus.columns = ['Tarifa', 'Descripción', 'Punta', 'Llano','Valle', 'P1', 'P3','Compensación Excedentes']
df_octopus['Empresa'] = "Octopus"
# Limpiamos nombres en Repsol y añadimos columnas faltantes
df_repsol.columns = ['Tarifa', 'Descripción', 'Punta', 'P1','P3']
df_repsol['Llano'] = df_repsol['Punta']
df_repsol['Valle'] = df_repsol['Punta']
df_repsol['Valle'][2] = 0.086415 # Durante 10 horas al día que se elijan.
df_repsol['Empresa'] = "Repsol"
#Limpiamos nombres en Xenera
df_xenera.columns = ['Valle', 'Llano', 'Punta', 'P1','Coste de gestión']
df_xenera['P1'] = 0.070357
df_xenera['P3'] = df_xenera['P1']
df_xenera['Tarifas'] = ["Home contigo","En casa","A vaquiña fija","Home contigo única"]
df_xenera['Empresa'] = "Xenera"

# Limpiamos nombres en Naturgy
df_naturgy = df_nat.transpose().reset_index()

df_naturgy.columns = ['Tarifa','Descripcion', 'A', 'B','Punta','P1','C','D']
df_naturgy = df_naturgy.drop(columns=['A', 'B','C','D']).drop(1)

df_naturgy['P3'] = df_naturgy['P1']
df_naturgy['Llano'] = df_naturgy['Punta']
df_naturgy['Valle'] = df_naturgy['Punta']

# Corregimos las potencias

valor_punta_pot = df_naturgy['P1'].str.extract(r'Punta: ([\d,]+)')
valor_valle_pot = df_naturgy['P3'].str.extract(r'Valle: ([\d,]+)')
df_naturgy['P1'] = valor_punta_pot
df_naturgy['P3'] = valor_valle_pot
# Corregimos la fila 0


valor_inicial = df_naturgy.loc[0, 'Punta'].str.extract(r'([\d,]+)')[0]

# Asigna este valor a las columnas 'Punta', 'Llano' y 'Valle' en la fila 0
df_naturgy.at[0, 'Punta'] = valor_inicial if valor_inicial else None
df_naturgy.at[0, 'Llano'] = valor_inicial if valor_inicial else None
df_naturgy.at[0, 'Valle'] = valor_inicial if valor_inicial else None

# Corregimos la fila 1
fila_1 = df_naturgy.loc[2, 'Punta']
valor_punta = re.search(r'Punta: ([\d,]+€/kWh)', fila_1)
valor_llano = re.search(r'Llano: ([\d,]+€/kWh)', fila_1)
valor_valle = re.search(r'Valle: ([\d,]+€/kWh)', fila_1)
df_naturgy.at[2, 'Punta'] = valor_punta.group(1) if valor_punta else None
df_naturgy.at[2, 'Llano'] = valor_llano.group(1) if valor_llano else None
df_naturgy.at[2, 'Valle'] = valor_valle.group(1) if valor_valle else None

lista = [df_naturgy,df_octopus,df_repsol,df_xenera]
for df in lista:
    print(df.columns)

df_naturgy.head()
#octopus = {
#    "nombre": "Octopus",
#    "punta": 0.1780, #€/kWh
#    "llano": 0.1300, #€/kWh
#    "valle": 0.0810, #€/kWh
#    "P1": 0.089, #€/kW/día
#    "P2": 0.026, #€/kW/día
#    "excedente": 0.05, #€/kWh
#    "batería": 0
#}

# Una vez limpitos, los metemos en un único data frame y lo guardamos en un csv
#df = pd.DataFrame(datos)
#df.to_csv("tarifas.csv", index=False)

Index(['Tarifa', 'Descripcion', 'Punta', 'P1', 'P3', 'Llano', 'Valle'], dtype='object')
Index(['Tarifa', 'Descripción', 'Punta', 'Llano', 'Valle', 'P1', 'P3',
       'Compensación Excedentes', 'Empresa'],
      dtype='object')
Index(['Tarifa', 'Descripción', 'Punta', 'P1', 'P3', 'Llano', 'Valle',
       'Empresa'],
      dtype='object')
Index(['Valle', 'Llano', 'Punta', 'P1', 'Coste de gestión', 'P3', 'Tarifas',
       'Empresa'],
      dtype='object')


C:\Users\Usuario\AppData\Local\Temp\ipykernel_16512\2049785649.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_repsol['Valle'][2] = 0.086415 # Durante 10 horas al día que se elijan.


,Tarifa,Descripcion,Punta,P1,P3,Llano,Valle
0,Por uso de Luz,La energía que consumes siempre al mismo prec...,"0,114900 €/kWh0,146137 €/kWh","0,103663","0,034042","0,114900 €/kWh0,146137 €/kWh","0,114900 €/kWh0,146137 €/kWh"
2,Noche Luz,Un precio diferente por cada tramo horario de ...,"0,170000€/kWh","0,103663","0,034042","0,116200€/kWh","0,082100€/kWh"
